## Continuous Pre-training using LLM recipie

In [1]:
! pip install unsloth datasets transformers torch tensorboard matplotlib

# Base Parameter Definition

## Settings

In [ ]:
# ====================================================IMPORT=================================================================

from datasets import DatasetDict, load_dataset, load_from_disk, Dataset

import os
import warnings
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import time
from torch.utils.tensorboard import SummaryWriter

warnings.filterwarnings("ignore")


from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
import torch
import pandas as pd
import sys
import os
from datasets import Dataset
import argparse

from utils.utils_train import pre_process, create_prompt, print_trainable_parameters
from unsloth import is_bfloat16_supported
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM


# ========================== CMD Argument Parser ==========================
# def parse_args():
#     parser = argparse.ArgumentParser(description="Train a model using CPT (Continual Pretraining Training)")
#     parser.add_argument("--per_device_train_batch_size", type=int, default=8, help="Batch size per device during training")
#     parser.add_argument("--per_device_eval_batch_size", type=int, default=8, help="Batch size per device during evaluation")
#     parser.add_argument("--src_lng", type=str, default="English", help="Source language default English")
#     parser.add_argument("--tgt_lng", type=str, default="Luxembourgish", help="Target language default Luxembourgish")
#     parser.add_argument("--num_train_epochs", type=int, default=1, help="Number of training epochs")
#     parser.add_argument("--learning_rate", type=float, default=1e-6, help="Learning rate for training")
#     parser.add_argument("--project_root", type=str, default="/Users/lujun.li/projects/mt_luxembourgish", help="Path to project root")
#     parser.add_argument("--training_dataset_path", type=str, default="data/processed/dataset_merged_llama_fake_targets.jsonl", help="Path to training dataset")
#     parser.add_argument("--model_path", type=str, default="/home/llama/Personal_Directories/srb/binary_classfication/Llama-3.2-3B-Instruct", help="Path to model")
#     parser.add_argument("--resume_from_checkpoint", type=bool, default=False, help="Resume training from checkpoint")
#     parser.add_argument("--resume_checkpoint_path", type=str, default=None, help="Path to checkpoint to resume training from")
#     parser.add_argument("--r", type=int, default=256, help="Number of random samples to be used for training")
#     parser.add_argument("--is_peft", type=bool, default=False, help="Use PEFT")
#     parser.add_argument("--is_unsloth", type=bool, default=False, help="Use UnSloth")
#     parser.add_argument("--is_train_response_only", type=bool, default=True, help="Train response only")
#     return parser.parse_args()

# args = parse_args()

# print("Arguments passed:")
# print(f"Train Batch Size: {args.per_device_train_batch_size}")
# print(f"Eval Batch Size: {args.per_device_eval_batch_size}")
# print(f"Number of Epochs: {args.num_train_epochs}")
# print(f"Learning Rate: {args.learning_rate}")
# print(f"Project Root: {args.project_root}")
# print(f"Training Dataset Path: {args.training_dataset_path}")
# print(f"Model path: {args.model_path}")
# print(f"tgt_lng: {args.tgt_lng}")
# print(f"src_lng: {args.src_lng}")
# print(f"Resume from checkpoint: {args.resume_from_checkpoint}")
# print(f"Resume checkpoint path: {args.resume_checkpoint_path}")
# print(f"r: {args.r}")
# print(f"is_peft: {args.is_peft}")
# print(f"is_unsloth: {args.is_unsloth}")
# print(f"is_train_response_only: {args.is_train_response_only}")

# learning_rate = args.learning_rate # Learning rate for the optimizer
# per_device_train_batch_size = args.per_device_train_batch_size  # Batch size for training per device
# per_device_eval_batch_size = args.per_device_eval_batch_size  # Batch size for evaluation per device
# num_train_epochs = args.num_train_epochs  # Number of epochs for training
# training_dataset_path = args.training_dataset_path
# project_root = args.project_root
# model_path = args.model_path
# resume_from_checkpoint = args.resume_from_checkpoint
# resume_checkpoint_path = args.resume_checkpoint_path
# src_lng = args.src_lng
# tgt_lng = args.tgt_lng
# r = args.r
# is_peft = args.is_peft
# is_unsloth = args.is_unsloth
# is_train_response_only = args.is_train_response_only

## Params need to be set
learning_rate = 1e-5 # Learning rate for the optimizer
per_device_train_batch_size = 10  # Batch size for training per device
per_device_eval_batch_size = 10  # Batch size for evaluation per device
num_train_epochs = 5  # Number of epochs for training
training_dataset_path = "data/training_dataset/dataset_GPT_split.jsonl"
project_root = "/home/snt/projects_lujun/mt_luxembourgish"
model_path = "/home/snt/llm_models/Llama-3.2-1B-Instruct"
resume_from_checkpoint = False
resume_checkpoint_path = None
src_lng = "English"
tgt_lng = "Luxembourgish"
r = 32
is_peft = False
is_unsloth = False
is_train_response_only = True


model_name = model_path.split("/")[-1]
train_ratio = 0.005  # Number of samples to be used for training and evaluation
warmup_ratio = 0.5
logging_steps = 10
evaluation_strategy="steps"
save_strategy="epoch"
eval_steps=10
max_grad_norm = 0.3
fp16 = not is_bfloat16_supported()
MAX_LEN = 512
weight_decay = 0.01
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
train_seed = 3407

if is_peft:
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",]
    lora_alpha = 8
    lora_dropout = 0
    random_state = 3407

current = time.time()
formatted_time = time.strftime("%m_%d_%H_%M", time.localtime(current))
if resume_from_checkpoint:
    output_dir = resume_checkpoint_path
else:
    if is_peft:
        input_file_name = training_dataset_path.split("/")[-1].split(".")[0]
        output_dir = f"logs/peft_{r}_{src_lng[:2]}_{tgt_lng[:2]}/fit_{formatted_time}_{train_ratio}_{input_file_name}"
    else:
        input_file_name = training_dataset_path.split("/")[-1].split(".")[0]
        output_dir = f"logs/full_{src_lng[:2]}_{tgt_lng[:2]}/fit_{formatted_time}_{train_ratio}_{input_file_name}"

if resume_from_checkpoint and resume_checkpoint_path is None:
    raise ValueError("Please provide a checkpoint path to resume training from")


/home/snt/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.5.1 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.10 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Data

In [ ]:
# ========================== dataset preparation ==========================

train_dataset_path = os.path.abspath(os.path.join(project_root, training_dataset_path))
sys.path.append(project_root)

train_dataset_df = pd.read_json(train_dataset_path, lines=True)
pre_processed_dataset_df = pre_process(train_dataset_df)

if not isinstance(pre_processed_dataset_df, pd.DataFrame):
    raise TypeError("data_preprocess should return a pandas DataFrame.")

dataset = Dataset.from_pandas(pre_processed_dataset_df)

# Filter by split
train_dataset = dataset.filter(lambda x: x["split"] == "train")
val_dataset = dataset.filter(lambda x: x["split"] == "val")

# Select subset
train_dataset = train_dataset.select(range(int(len(train_dataset) * train_ratio)))
val_dataset = val_dataset.select(range(int(len(val_dataset) * train_ratio)))  # Avoid out-of-range error

# Rename columns
train_dataset = train_dataset.rename_columns({
    "input": "Luxembourgish",
    "translated_text": "English",
})

val_dataset = val_dataset.rename_columns({
    "input": "Luxembourgish",
    "translated_text": "English",
})

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = train_dataset.map(
    lambda sample: {
        "full_prompt": create_prompt(sample, src_lng=src_lng, tgt_lng=tgt_lng, mode="train", tokenizer=tokenizer)["full_prompt"]
    }
).select_columns(["full_prompt"])

val_dataset = val_dataset.map(
    lambda sample: {
        "full_prompt": create_prompt(sample, src_lng=src_lng, tgt_lng=tgt_lng, mode="train", tokenizer=tokenizer)["full_prompt"]
    }
).select_columns(["full_prompt"])

def tokenize_function(examples):
    return tokenizer(
        examples["full_prompt"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_tensors="pt",
    )

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["full_prompt"])
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["full_prompt"])

Length of inputs before: 207011
Length of inputs after: 204642
Removed rows: 2369


Map: 100%|██████████| 18/18 [00:00<00:00, 1682.81 examples/s]


## Load the models using unsloth or huggingface

In [4]:
from unsloth import FastLanguageModel
import torch
from peft import get_peft_model, LoraConfig


# Using Unsloth Acceleration 
if is_unsloth:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = f"unsloth/{model_name}", # or choose "unsloth/Llama-3.2-1B-Instruct"
        # model_name = model_path,
        max_seq_length = MAX_LEN,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    
    if is_peft:
        model = FastLanguageModel.get_peft_model(
            model,
            r = r, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules = target_modules,
            lora_alpha = lora_alpha,
            lora_dropout = lora_dropout, # Supports any, but = 0 is optimized
            bias = "none",    # Supports any, but = "none" is optimized
            # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
            use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
            random_state = random_state,
            use_rslora = False,  # We support rank stabilized LoRA
            loftq_config = None, # And without LoftQ
        )

# Using transformer huggingface   
else:
    model = AutoModelForCausalLM.from_pretrained(model_path)
    model.config.use_cache = False
    if is_peft:
        lora_config = LoraConfig(
            r=r, 
            target_modules=target_modules, 
            lora_alpha=lora_alpha, 
            lora_dropout=lora_dropout, 
            bias="none", 
            random_state=random_state,
            use_rslora=False,
            loftq_config=None  # And without LoftQ
        )
        model = get_peft_model(model, lora_config)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

print (model)
print(print_trainable_parameters(model))

for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Shape: {param.shape}, Trainable parameters: {param.numel()}")
    else:
        print(f"Layer: {name}, Shape: {param.shape}, Non-trainable parameters: {param.numel()}")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [7]:
from transformers import logging as transformers_logging
from accelerate import Accelerator
from unsloth.chat_templates import train_on_responses_only
import warnings
warnings.simplefilter("ignore")
transformers_logging.set_verbosity_error()

def train_ddp_accelerate_sft():
    training_args = SFTConfig(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        warmup_ratio=warmup_ratio,
        evaluation_strategy=evaluation_strategy,
        save_strategy=save_strategy,
        logging_steps=logging_steps,
        eval_steps=eval_steps,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16= fp16,
        bf16 = is_bfloat16_supported(),
        max_grad_norm=max_grad_norm,
        group_by_length=True,
        lr_scheduler_type="cosine",
        report_to="tensorboard",
        remove_unused_columns=False,
        disable_tqdm=False,
        seed = train_seed,
        ddp_find_unused_parameters=False, # Avoids warnings
        dataloader_num_workers=4,  # Adjust number of workers based on hardware
        # load_best_model_at_end=True,
    )

    trainer = SFTTrainer(
        model = model,
        tokenizer=tokenizer,
        train_dataset = tokenized_train_dataset,
        eval_dataset = tokenized_val_dataset,
        dataset_text_field = "full_prompt",
        max_seq_length = MAX_LEN,
        data_collator = data_collator,
        dataset_num_proc = 2,
        packing = False, # Can make training 5x faster for short sequences.
        args = training_args,
    )

    if is_train_response_only:
        trainer = train_on_responses_only(
            trainer,
            instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
            response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
        )
    trainer_stats = trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    print("Finished training SFT.")
    return trainer_stats


In [8]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A6000. Max memory = 47.536 GB.
0.0 GB of memory reserved.


In [9]:
trainer_stats = None

def main():
    trainer_stats = train_ddp_accelerate_sft()
    return trainer_stats

if __name__ == "__main__":
    trainer_stats = main()

Converting train dataset to ChatML (num_proc=2): 100%|██████████| 1005/1005 [00:00<00:00, 2317.94 examples/s]
Applying chat template to train dataset (num_proc=2): 100%|██████████| 1005/1005 [00:01<00:00, 759.40 examples/s] 
Converting eval dataset to ChatML (num_proc=2): 100%|██████████| 18/18 [00:00<00:00, 58.94 examples/s]
Applying chat template to eval dataset (num_proc=2): 100%|██████████| 18/18 [00:01<00:00, 13.96 examples/s]
Map: 100%|██████████| 18/18 [00:00<00:00, 2256.35 examples/s]


Step,Training Loss,Validation Loss
10,6.676900,5.382022
20,2.141100,1.283134
30,1.030200,0.994903
40,0.811500,0.868839
50,0.713400,0.774849
60,0.597600,0.726333
70,0.486400,0.696730
80,0.444500,0.677242
90,0.377500,0.678876
100,0.310000,0.675392


Finished training SFT.


In [10]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

447.7424 seconds used for training.
7.46 minutes used for training.
Peak reserved memory = 33.484 GB.
Peak reserved memory for training = 33.484 GB.
Peak reserved memory % of max memory = 70.439 %.
Peak reserved memory for training % of max memory = 70.439 %.


In [12]:
from unsloth.chat_templates import get_chat_template

system_message = f"You are a helpful AI assistant for translation."
input_text = (
    "Translate the English input text into Luxembourgish. Do not include any additional information or unrelated content.\n\n"
    "The concept of the lead candidate is not without controversy: One pro-argument is that the election becomes more democratic this way. "
    "On the other hand, the heads of state and government lose power as a result. Simply put, the EU is responsible for what is agreed upon between it and the member states, "
    "for example, for the internal market, trade policy, monetary policy, and competition rules, and there is also a common security and foreign policy."
)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/snt/projects_lujun/mt_luxembourgish/logs/full_En_Lu/fit_03_01_22_42_0.005_dataset_GPT_split/checkpoint-102",
    max_seq_length = MAX_LEN,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": input_text},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = MAX_LEN, use_cache = True,
                         temperature = 1.0)
print (tokenizer.batch_decode(outputs)[0])

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.536 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
/home/snt/projects_lujun/mt_luxembourgish/logs/full_En_Lu/fit_03_01_22_42_0.005_dataset_GPT_split/checkpoint-102 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are a helpful AI assistant for translation.<|eot_id|><|start_header_id|>user<|end_header_id|>

Translate the English input text into Luxembourgish. Do not include any additional information or unrelated content.

The concept 

In [17]:
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, pipeline
import torch


device = "cuda:0"
model_checkpoint = "/home/snt/projects_lujun/mt_luxembourgish/logs/En_Lu/fit_02_26_02_15_0.001_dataset_GPT_split/checkpoint-609"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
system_message = f"You are a helpful AI assistant for translation."
input_text = """The concept of the lead candidate is not without controversy: One pro-argument is that the election becomes more democratic this way. On the other hand, the heads of state and government lose power as a result. Simply put, the EU is responsible for what is agreed upon between it and the member states, for example, for the internal market, trade policy, monetary policy, and competition rules, and there is also a common security and foreign policy."""
eos_token = tokenizer.eos_token

# Construct the full prompt.
full_prompt = (
    "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n" + system_message + "<|eot_id|>" +  # System Message
    "<|start_header_id|>user<|end_header_id|>\n\n" + f"Translate the English input text into Luxembourgish. Do not include any additional information or unrelated content.\n\n{input_text}"  +  "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" # User Query
)

# full_prompt = f"Translate the English input text into Luxembourgish.\n{input_text}. Do not include any additional information or unrelated content."

# Tokenize input text
encoded_input = tokenizer(full_prompt, return_tensors="pt", add_special_tokens=True)
model_inputs = encoded_input.to(device)
generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        temperature=1.0,
    )
decoded_output = tokenizer.batch_decode(generated_ids)[0].replace(full_prompt, "")

special_tokens = tokenizer.all_special_tokens
for token in special_tokens:
    decoded_output = decoded_output.replace(token, "")
    
print("Test Output:\n", decoded_output)


Test Output:
 

De Konzept vum éissten Zweeten Kandidaten ass net ohne Kämpft op de Leschter: E Pro Argument géif d’Wahl mä méi démokrateesch déi Weidergehalten. An deem Zuel géif den Éischte Exekutiv-Residenten an den Regierungsresidenten op d’Spill gréisst. Esou geet einfach, déi EU géif déi fir deenen am Vertrag mat de Membrancer ënnerkleeften ëm Beilechen. Zum Beispill fëren d'Mënschent, d'Traderpolt, d'Monetairpolt, d'Konkurrenzvirschrëfte an d'Zerwéierung vun der EU.


In [19]:
print (full_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for translation.<|eot_id|><|start_header_id|>user<|end_header_id|>

Translate the English input text into Luxembourgish. Do not include any additional information or unrelated content.

The concept of the lead candidate is not without controversy: One pro-argument is that the election becomes more democratic this way. On the other hand, the heads of state and government lose power as a result. Simply put, the EU is responsible for what is agreed upon between it and the member states, for example, for the internal market, trade policy, monetary policy, and competition rules, and there is also a common security and foreign policy.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
